# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import sys

sys.setrecursionlimit(10**6)

# Import API keys
from config import wkey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'config'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [ ]:
citylist = []
cloudiness= []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

units = "metric"
base_url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{base_url}appid={wkey}&units={units}&q="
line = "-----------------------------"

print("Beginning Data Retrieval")
print(line)
count = 0
for city in cities:
    try:
        response = requests.get(query_url + city).json()
        print(f"Processing Record {count} / {city}")
        citylist.append(response['name'])
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        count += 1
    except:
        print("City not found. Skipping...")
        cities.remove(city)
        count += 1
        pass
print(line)
print("Data Retrieval Complete")
print(line)


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_data_dict = {
    'City':citylist, "Cloudiness":cloudiness, "Country":country, "Date":date, "Humidity":humidity,
    "Lat":lat, "Lng":lng, "Max Temp":max_temp, "Wind Speed":wind_speed
}

city_data_df = pd.DataFrame(city_data_dict)

city_data_df.to_csv(output_data_file)

In [ ]:
city_data_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# trying to get the unitdate to real date.

date = pd.to_datetime(city_data_df['Date'], unit='ms')
date

In [ ]:
lat_temp_plot = city_data_df.plot.scatter(y="Max Temp", x="Lat", grid=True)
plt.ylabel("Maximum Temperature (degrees C)")
plt.xlabel('Latitude (degrees)')
plt.title(f'City Latitude vs Maximum Temperature')
# date = pd.to_datetime(city_data_df['Date'].head(1))
plt.show()
plt.savefig('output_data/cit-vs-temp.png')

#### Latitude vs. Humidity Plot

In [ ]:
lat_hum_plot = city_data_df.plot.scatter(y='Humidity', x='Lat', grid=True)
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude (degrees)")
plt.title("City Latitude vs. Humidity")
plt.show()
plt.savefig('output_data/cit-vs-humidity.png')

#### Latitude vs. Cloudiness Plot

In [ ]:
lat_clouds_plot = city_data_df.plot.scatter(y='Cloudiness', x='Lat', grid=True)
plt.ylabel("Cloudiness (%)")
plt.xlabel('Latitude (degrees)')
plt.title("City Latitude vs. Cloudiness")
plt.show()
plt.savefig('output_data/cit-vs-coudiness.png')

#### Latitude vs. Wind Speed Plot

In [ ]:
lat_clouds_plot = city_data_df.plot.scatter(y='Wind Speed', x='Lat', grid=True)
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude (degrees)")
plt.title("City Latitude vs. Wind Speed")
plt.show()
plt.savefig('output_data/cit-vs-wind.png')

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots
def line_regress_function(x_values, y_values):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = f"y = {round(slope,2)}x + {round(intercept,2)}"
    plt.scatter(x_values, y_values)
    plt.grid()
    plt.plot(x_values, regress_values, "r-")
    return(plt)

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_hem_df = city_data_df[city_data_df["Lat"]>0]
southern_hem_df = city_data_df[city_data_df["Lat"]<0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
line_regress_function(northern_hem_df['Lat'],northern_hem_df["Max Temp"])
plt.ylabel("Maximum Temperature (degrees C)")
plt.xlabel("Latitude (degrees)")
plt.title("Northern Hemisphere - City Latitude vs. Maximum Temperature")
plt.show()
plt.savefig('output_data/Northern-Hemisphere-cit-vs-temp-regress.png')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
line_regress_function(southern_hem_df['Lat'],southern_hem_df["Max Temp"])
plt.ylabel("Maximum Temperature (degrees C)")
plt.xlabel("Latitude (degrees)")
plt.title("Southern Hemisphere - City Latitude vs. Maximum Temperature")
plt.show()
plt.savefig('output_data/Southern-Hemisphere-cit-vs-temp-regress.png')

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
line_regress_function(northern_hem_df['Lat'],northern_hem_df["Humidity"])
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude (degrees)")
plt.title("Northern Hemisphere - City Latitude vs. Humidity")
plt.show()
plt.savefig('output_data/Northern-Hemisphere-cit-vs-humidity-regress.png')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
line_regress_function(southern_hem_df['Lat'],southern_hem_df["Humidity"])
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude (degrees)")
plt.title("Southern Hemisphere - City Latitude vs. Humidity")
plt.show()
plt.savefig('output_data/Southern-Hemisphere-cit-vs-humidity-regress.png')

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
line_regress_function(northern_hem_df['Lat'],northern_hem_df["Cloudiness"])
plt.ylabel("Cloudiness (%)")
plt.xlabel('Latitude (degrees)')
plt.title("Northern Hemisphere - City Latitude vs. Cloudiness")
plt.show()
plt.savefig('output_data/Northern-Hemisphere-cit-vs-coludiness-regress.png')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
line_regress_function(southern_hem_df['Lat'],southern_hem_df["Cloudiness"])
plt.ylabel("Cloudiness (%)")
plt.xlabel('Latitude (degrees)')
plt.title("Southern Hemisphere - City Latitude vs. Cloudiness")
plt.show()
plt.savefig('output_data/Southern-Hemisphere-cit-vs-coludiness-regress.png')

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
line_regress_function(northern_hem_df['Lat'],northern_hem_df["Wind Speed"])
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude (degrees)")
plt.title("Northern Hemisphere - City Latitude vs. Wind Speed")
plt.show()
plt.savefig('output_data/Northern-Hemisphere-cit-vs-wind-regress.png')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
line_regress_function(southern_hem_df['Lat'],southern_hem_df["Wind Speed"])
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude (degrees)")
plt.title("Southern Hemisphere - City Latitude vs. Wind Speed")
plt.show()
plt.savefig('output_data/Southern-Hemisphere-cit-vs-wind-regress.png')